## General Stats

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

# Load datasets
matches = pd.read_csv("../ipl_wrang/ingestion/cleaned_data/matches_cleaned.csv")
deliveries = pd.read_csv("../ipl_wrang/ingestion/cleaned_data/deliveries_cleaned.csv")
ipl_runs = pd.read_csv("../ipl_wrang/data/IPL_last_year_runs.csv")
ipl_wickets = pd.read_csv("../ipl_wrang/data/IPL_last_year_wickets.csv")


### Impact of Toss on matches

In [2]:

matches['toss_win'] = matches['toss_winner'] == matches['winner']
toss_impact = matches['toss_win'].value_counts().reset_index()
toss_impact.columns = ['Toss_Winner_Won', 'Count']
toss_impact['Toss_Winner_Won'] = toss_impact['Toss_Winner_Won'].map({True: 'Toss Win = Match Win', False: 'Toss Win ≠ Match Win'})

fig = px.bar(toss_impact, x='Toss_Winner_Won', y='Count', color='Toss_Winner_Won', title="Impact of Toss on Match Result", text_auto=True)
fig.show()


### Head to head between teams

In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

# Load matches dataset
# matches = pd.read_csv("matches.csv")

# Unique list of teams
teams = sorted(pd.unique(matches[['team1', 'team2']].values.ravel('K')))
teams = [t for t in teams if pd.notna(t)]

# Precompute H2H for all pairs
all_h2h = {}
for t1 in teams:
    for t2 in teams:
        if t1 != t2:
            h2h = matches[((matches['team1'] == t1) & (matches['team2'] == t2)) |
                          ((matches['team1'] == t2) & (matches['team2'] == t1))]
            rivalry = h2h['winner'].value_counts().reset_index()
            rivalry.columns = ['Team', 'Wins']
            all_h2h[(t1, t2)] = rivalry

# Create all matchup options
matchups = [f"{t1} vs {t2}" for t1 in teams for t2 in teams if t1 != t2]

# Mapping for easy lookup
matchup_mapping = {f"{t1} vs {t2}": (t1, t2) for t1 in teams for t2 in teams if t1 != t2}

# Initial matchup
initial_matchup = list(matchup_mapping.keys())[0]
t1_init, t2_init = matchup_mapping[initial_matchup]
rivalry = all_h2h.get((t1_init, t2_init), all_h2h.get((t2_init, t1_init)))

# Create figure
fig = go.Figure()
fig.add_trace(go.Bar(x=rivalry['Team'], y=rivalry['Wins']))

# Dropdown options
buttons = []
for matchup in matchup_mapping.keys():
    t1, t2 = matchup_mapping[matchup]
    df = all_h2h.get((t1, t2), all_h2h.get((t2, t1)))
    if df is not None:
        buttons.append(dict(label=matchup,
                            method="update",
                            args=[{"x": [df['Team']], "y": [df['Wins']]},
                                  {"title": f"Head to Head: {matchup}"}]))

# Layout
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
            x=0.5,
            xanchor="center",
            y=1.15,
            yanchor="top"
        )
    ],
    title={"text": f"Head to Head: {initial_matchup}", "y":0.9, "x":0.5, "xanchor": "center", "yanchor": "top"},
    xaxis_title="Team",
    yaxis_title="Wins",
    showlegend=False,
    margin=dict(t=100)
)

fig.show()

### Average margin in winning

In [4]:

avg_runs_season = matches.groupby('season')['result_margin'].mean().reset_index()

fig = px.line(avg_runs_season, x='season', y='result_margin', title='Average Winning Margin per Season')
fig.update_layout(xaxis_title="Season", yaxis_title="Average Result Margin")
fig.show()


### How each team wins

In [5]:

teams = matches['winner'].dropna().unique()

figures = {}
for team in teams:
    filtered_df = matches[matches['winner'] == team]
    fig = px.histogram(
        filtered_df,
        x="result",
        title=f"Results when {team} wins",
        text_auto=True
    )
    fig.update_layout(xaxis_title="Result", yaxis_title="Count")
    figures[team] = fig

buttons = []
for team in teams:
    visible_array = [t == team for t in teams]
    buttons.append(dict(label=team, method="update", args=[{"visible": visible_array}, {"title": f"Results when {team} wins"}]))

fig = go.Figure()
for team in teams:
    fig.add_trace(figures[team].data[0])

for i, trace in enumerate(fig.data):
    trace.visible = (i == 0)

fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons, x=1.15, y=1.2)],
    title=f"Results when {teams[0]} wins",
)

fig.show()


### Venue stats

In [6]:
venues = matches['venue'].dropna().unique()

figures = {}
for venue in venues:
    filtered_df = matches[matches['venue'] == venue]
    fig = px.histogram(
        filtered_df,
        x="result",
        title=f"Match Results at {venue}",
        text_auto=True
    )
    fig.update_layout(xaxis_title="Result", yaxis_title="Count")
    figures[venue] = fig

buttons = []
for venue in venues:
    visible_array = [v == venue for v in venues]
    buttons.append(dict(label=venue, method="update", args=[{"visible": visible_array}, {"title": f"Match Results at {venue}"}]))

fig = go.Figure()
for venue in venues:
    fig.add_trace(figures[venue].data[0])

for i, trace in enumerate(fig.data):
    trace.visible = (i == 0)

fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons, x=1.15, y=1.2)],
    title=f"Match Results at {venues[0]}",
)

fig.show()